# XplainML Demo - Interpretable Machine Learning

This notebook demonstrates the capabilities of XplainML, a comprehensive tool for interpretable machine learning on tabular data.

## Table of Contents
1. [Setup and Imports](#Setup-and-Imports)
2. [Data Generation and Preprocessing](#Data-Generation-and-Preprocessing)
3. [Model Training and Comparison](#Model-Training-and-Comparison)
4. [Model Explanations](#Model-Explanations)
5. [Visualizations](#Visualizations)
6. [Making Predictions](#Making-Predictions)
7. [Advanced Features](#Advanced-Features)

---

## 1. Setup and Imports

First, let's import all necessary libraries and set up our environment.

In [ ]:
# System imports
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add backend directory to path
backend_dir = os.path.join('..', 'backend')
if backend_dir not in sys.path:
    sys.path.append(backend_dir)

# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# XplainML modules from backend
from data_preprocessing import DataPreprocessor, create_sample_data
from models import MLModel, ModelTuner, compare_models
from prediction import Predictor, create_prediction_report
from explainer import ModelExplainer
from visualizer import ModelVisualizer

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("✅ All modules imported successfully!")
print("🚀 XplainML Demo Ready!")

## 2. Data Generation and Preprocessing

Let's start by generating sample data and exploring XplainML's preprocessing capabilities.

In [ ]:
# Generate sample dataset
print("📊 Generating sample dataset...")

# Create data directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

# Generate sample data
sample_df = create_sample_data('../data/demo_data.csv', n_samples=1500)

print(f"Dataset created with shape: {sample_df.shape}")
print(f"Columns: {list(sample_df.columns)}")

# Display first few rows
sample_df.head(10)

In [ ]:
# Explore the data
print("🔍 Data Exploration")
print("=" * 40)

# Basic statistics
print("\n📈 Data Info:")
print(sample_df.info())

print("\n📊 Summary Statistics:")
sample_df.describe()

In [ ]:
# Check for missing values
print("❓ Missing Values:")
missing_values = sample_df.isnull().sum()
print(missing_values[missing_values > 0])

# Visualize missing values
if missing_values.sum() > 0:
    plt.figure(figsize=(10, 6))
    missing_df = sample_df.isnull().sum().reset_index()
    missing_df.columns = ['Column', 'Missing_Count']
    missing_df = missing_df[missing_df['Missing_Count'] > 0]
    
    sns.barplot(data=missing_df, x='Column', y='Missing_Count')
    plt.title('Missing Values by Column')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("✅ No missing values found!")

In [ ]:
# Initialize preprocessor and run full pipeline
print("🔧 Running Data Preprocessing Pipeline...")
print("=" * 50)

preprocessor = DataPreprocessor()

# Run comprehensive preprocessing
data = preprocessor.preprocess_pipeline(
    file_path='../data/demo_data.csv',
    target_column='target',
    test_size=0.2,
    missing_strategy='mean',
    encoding_type='auto',
    scaling_type='standard',
    random_state=42
)

print(f"\n✅ Preprocessing completed!")
print(f"📋 Task Type: {data['task_type']}")
print(f"🎯 Target: {data['target_name']}")
print(f"📊 Features: {len(data['feature_names'])}")
print(f"🏋️ Training samples: {data['X_train'].shape[0]}")
print(f"🧪 Test samples: {data['X_test'].shape[0]}")

## 3. Model Training and Comparison

Now let's train different models and compare their performance.

In [ ]:
# Train a single Random Forest model
print("🤖 Training Random Forest Model...")
print("=" * 40)

# Initialize and train model
rf_model = MLModel('random_forest', data['task_type'])
rf_model.fit(data['X_train'], data['y_train'])

# Evaluate performance
train_metrics = rf_model.evaluate(data['X_train'], data['y_train'])
test_metrics = rf_model.evaluate(data['X_test'], data['y_test'])

print(f"\n📊 Training Metrics:")
for metric, value in train_metrics.items():
    print(f"  {metric}: {value:.4f}")

print(f"\n🎯 Test Metrics:")
for metric, value in test_metrics.items():
    print(f"  {metric}: {value:.4f}")

In [ ]:
# Compare multiple models
print("⚖️ Comparing Multiple Models...")
print("=" * 40)

# Run model comparison
comparison_results = compare_models(
    data['X_train'], data['y_train'],
    data['X_test'], data['y_test'],
    task_type=data['task_type']
)

# Create comparison DataFrame
comparison_data = []
for model_name, result in comparison_results.items():
    if 'test_metrics' in result:
        row = {'Model': model_name}
        row.update(result['test_metrics'])
        comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)
print("\n📊 Model Comparison Results:")
comparison_df

In [ ]:
# Visualize model comparison
plt.figure(figsize=(12, 6))

if data['task_type'] == 'classification':
    metric = 'accuracy'
    title = 'Model Accuracy Comparison'
else:
    metric = 'r2'
    title = 'Model R² Score Comparison'

sns.barplot(data=comparison_df, x='Model', y=metric)
plt.title(title)
plt.ylabel(metric.title())
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Select best model
best_idx = comparison_df[metric].idxmax()
best_model_name = comparison_df.loc[best_idx, 'Model']
best_score = comparison_df.loc[best_idx, metric]

print(f"\n🏆 Best Model: {best_model_name}")
print(f"📊 Best {metric}: {best_score:.4f}")

# Use the best model for further analysis
best_model = comparison_results[best_model_name]['model']

In [ ]:
# Hyperparameter tuning example
print("⚙️ Hyperparameter Tuning Example...")
print("=" * 45)

# Initialize tuner
tuner = ModelTuner('random_forest', data['task_type'], search_type='grid')

# Perform tuning (using smaller parameter space for demo)
tuned_model = tuner.tune(data['X_train'], data['y_train'], cv=3)

print(f"\n🎯 Best Parameters: {tuner.best_params}")
print(f"📊 Best CV Score: {tuner.best_score:.4f}")

# Evaluate tuned model
tuned_test_metrics = tuned_model.evaluate(data['X_test'], data['y_test'])
print(f"\n🧪 Tuned Model Test Metrics:")
for metric, value in tuned_test_metrics.items():
    print(f"  {metric}: {value:.4f}")

## 4. Model Explanations

Now let's generate comprehensive explanations for our best model.

In [ ]:
# Initialize explainer
print("🔍 Initializing Model Explainer...")
print("=" * 40)

explainer = ModelExplainer(
    model=best_model,
    X_train=data['X_train'],
    y_train=data['y_train'],
    feature_names=data['feature_names']
)

print("✅ Explainer initialized successfully!")

In [ ]:
# Generate global explanations
print("🌍 Generating Global Explanations...")
print("=" * 40)

global_explanations = explainer.explain_global(method='all', max_display=15)

# Display results
for method, explanation in global_explanations.items():
    if 'feature_importance' in explanation:
        print(f"\n📊 {method.upper()} Feature Importance:")
        importance = explanation['feature_importance']
        
        # Show top 10 features
        for i, (feature, score) in enumerate(list(importance.items())[:10]):
            print(f"  {i+1:2d}. {feature:20s}: {score:.4f}")

In [ ]:
# Generate local explanations for individual samples
print("🎯 Generating Local Explanations...")
print("=" * 40)

# Select a few samples for explanation
sample_indices = [0, 1, 2]

local_explanations = []
for idx in sample_indices:
    sample = data['X_test'].iloc[idx]
    explanation = explainer.explain_local(sample, method='all', num_features=8)
    local_explanations.append(explanation)
    
    print(f"\n🔍 Sample {idx + 1} Explanation:")
    
    # Show prediction info
    if 'prediction_info' in explanation:
        pred_info = explanation['prediction_info']
        print(f"  Prediction: {pred_info['prediction']}")
        
        if 'probabilities' in pred_info:
            probs = pred_info['probabilities']
            print(f"  Probabilities: {probs}")
    
    # Show SHAP contributions
    if 'shap' in explanation:
        contributions = explanation['shap']['feature_contributions']
        print(f"  Top SHAP Contributions:")
        for i, (feature, contrib) in enumerate(list(contributions.items())[:5]):
            direction = "↗️" if contrib > 0 else "↘️"
            print(f"    {direction} {feature}: {contrib:.4f}")

In [ ]:
# Compare explanation methods
print("⚖️ Comparing Explanation Methods...")
print("=" * 40)

# Use first test sample
sample = data['X_test'].iloc[0]
comparison = explainer.compare_explanations(sample, methods=['shap', 'lime'])

if 'comparison' in comparison:
    for comp_name, comp_data in comparison['comparison'].items():
        print(f"\n🔄 {comp_name}:")
        print(f"  Overlap Ratio: {comp_data['overlap_ratio']:.3f}")
        print(f"  Common Features: {comp_data['common_features']}")
        
        if comp_data['unique_to_method1']:
            method1 = comp_name.split('_vs_')[0]
            print(f"  Unique to {method1}: {comp_data['unique_to_method1']}")
        
        if comp_data['unique_to_method2']:
            method2 = comp_name.split('_vs_')[1]
            print(f"  Unique to {method2}: {comp_data['unique_to_method2']}")

## 5. Visualizations

Let's create comprehensive visualizations to understand our model better.

In [ ]:
# Initialize visualizer
print("📊 Initializing Visualizer...")
visualizer = ModelVisualizer(best_model, explainer)

print("✅ Visualizer ready!")

In [ ]:
# Plot feature importance
print("📊 Feature Importance Visualization")

# Get feature importance from best model
importance = best_model.get_feature_importance()

if importance:
    # Create matplotlib plot
    features = list(importance.keys())[:12]
    values = list(importance.values())[:12]
    
    plt.figure(figsize=(12, 8))
    y_pos = np.arange(len(features))
    
    plt.barh(y_pos, values[::-1], alpha=0.8)
    plt.yticks(y_pos, features[::-1])
    plt.xlabel('Feature Importance Score')
    plt.title('Top 12 Feature Importance')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Feature importance not available for this model type")

In [ ]:
# Plot prediction distribution
print("📈 Prediction Distribution")

predictions = best_model.predict(data['X_test'])
y_true = data['y_test']

plt.figure(figsize=(15, 5))

if data['task_type'] == 'classification':
    # Classification: bar plots
    plt.subplot(1, 3, 1)
    unique_pred, counts_pred = np.unique(predictions, return_counts=True)
    plt.bar(range(len(unique_pred)), counts_pred, alpha=0.7, label='Predictions')
    plt.xticks(range(len(unique_pred)), unique_pred)
    plt.title('Prediction Distribution')
    plt.xlabel('Class')
    plt.ylabel('Count')
    
    plt.subplot(1, 3, 2)
    unique_true, counts_true = np.unique(y_true, return_counts=True)
    plt.bar(range(len(unique_true)), counts_true, alpha=0.7, color='orange', label='True Values')
    plt.xticks(range(len(unique_true)), unique_true)
    plt.title('True Distribution')
    plt.xlabel('Class')
    plt.ylabel('Count')
    
    # Confusion Matrix
    plt.subplot(1, 3, 3)
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_true, predictions)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
else:
    # Regression: histograms and scatter plot
    plt.subplot(1, 3, 1)
    plt.hist(predictions, alpha=0.7, bins=20, label='Predictions')
    plt.hist(y_true, alpha=0.7, bins=20, label='True Values')
    plt.title('Distribution Comparison')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.legend()
    
    plt.subplot(1, 3, 2)
    plt.scatter(y_true, predictions, alpha=0.6)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('Predictions vs True Values')
    
    plt.subplot(1, 3, 3)
    residuals = y_true - predictions
    plt.scatter(predictions, residuals, alpha=0.6)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predictions')
    plt.ylabel('Residuals')
    plt.title('Residual Plot')

plt.tight_layout()
plt.show()

In [ ]:
# Partial Dependence Plots
print("📊 Partial Dependence Plots")

# Select top 4 important features for PDP
if importance:
    top_features = list(importance.keys())[:4]
    
    plt.figure(figsize=(15, 10))
    
    for i, feature in enumerate(top_features):
        plt.subplot(2, 2, i + 1)
        
        # Simple partial dependence calculation
        feature_values = data['X_test'][feature]
        
        if feature_values.dtype in ['int64', 'float64']:
            eval_values = np.linspace(feature_values.min(), feature_values.max(), 20)
        else:
            eval_values = feature_values.unique()[:10]
        
        partial_deps = []
        base_instance = data['X_test'].iloc[0].copy()
        
        for val in eval_values:
            base_instance[feature] = val
            pred = best_model.predict(pd.DataFrame([base_instance]))[0]
            partial_deps.append(pred)
        
        plt.plot(eval_values, partial_deps, 'o-', linewidth=2, markersize=4)
        plt.xlabel(feature)
        plt.ylabel('Prediction')
        plt.title(f'Partial Dependence: {feature}')
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Feature importance not available for PDP generation")

## 6. Making Predictions

Let's see how to use our trained model for making predictions on new data.

In [ ]:
# Create predictor
print("🎯 Setting up Predictor...")
predictor = Predictor(best_model, preprocessor)

print("✅ Predictor ready!")

In [ ]:
# Make predictions on test set
print("📊 Making Predictions on Test Set...")

prediction_result = predictor.predict(
    data['X_test'], 
    return_probabilities=True, 
    include_confidence=True
)

print(f"✅ Predictions completed for {len(prediction_result['predictions'])} samples")
print(f"📊 Model type: {prediction_result['model_type']}")
print(f"🎯 Task type: {prediction_result['task_type']}")

# Show sample predictions
print(f"\n🔍 Sample Predictions:")
for i in range(5):
    pred = prediction_result['predictions'][i]
    print(f"  Sample {i+1}: {pred}")
    
    if 'probabilities' in prediction_result:
        probs = prediction_result['probabilities'][i]
        print(f"    Probabilities: {probs}")
    
    if 'confidence' in prediction_result:
        conf = prediction_result['confidence'][i]
        print(f"    Confidence: {conf:.3f}")
    print()

In [ ]:
# Single prediction example
print("🎯 Single Prediction Example...")

# Create a sample instance
sample_instance = data['X_test'].iloc[0].to_dict()

print(f"📋 Input features:")
for feature, value in sample_instance.items():
    print(f"  {feature}: {value}")

# Make single prediction
single_result = predictor.predict_single(
    sample_instance, 
    return_probabilities=True, 
    include_confidence=True
)

print(f"\n🎯 Prediction Results:")
print(f"  Prediction: {single_result['prediction']}")
print(f"  Model: {single_result['model_type']}")
print(f"  Task: {single_result['task_type']}")

if 'probabilities' in single_result:
    print(f"  Probabilities: {single_result['probabilities']}")

if 'confidence' in single_result:
    print(f"  Confidence: {single_result['confidence']:.3f}")

In [ ]:
# Generate comprehensive prediction report
print("📋 Generating Prediction Report...")

report = create_prediction_report(
    predictor, 
    data['X_test'], 
    data['y_test']
)

print(f"\n📊 Prediction Summary:")
summary = report['prediction_summary']
for key, value in summary.items():
    if key != 'prediction_time':
        print(f"  {key}: {value}")

print(f"\n🎯 Evaluation Metrics:")
if 'evaluation_metrics' in report:
    for metric, value in report['evaluation_metrics'].items():
        print(f"  {metric}: {value:.4f}")

if 'confidence_stats' in report:
    print(f"\n🔒 Confidence Statistics:")
    conf_stats = report['confidence_stats']
    for key, value in conf_stats.items():
        print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")

## 7. Advanced Features

Let's explore some advanced features of XplainML.

In [ ]:
# Feature interactions analysis
print("🔗 Analyzing Feature Interactions...")

interactions = explainer.analyze_feature_interactions(sample_size=200)

if isinstance(interactions, dict) and 'top_interactions' in interactions:
    print(f"\n🔝 Top Feature Interactions:")
    for i, interaction in enumerate(interactions['top_interactions'][:8]):
        feature1 = interaction['feature1']
        feature2 = interaction['feature2']
        strength = interaction['interaction_strength']
        print(f"  {i+1:2d}. {feature1} ↔ {feature2}: {strength:.4f}")
else:
    print(f"⚠️ {interactions}")

In [ ]:
# Model saving and loading
print("💾 Model Saving and Loading...")

# Save the model
model_path = '../data/best_model.pkl'
best_model.save_model(model_path)
print(f"✅ Model saved to {model_path}")

# Load the model
loaded_model = MLModel.load_model(model_path)
print(f"✅ Model loaded from {model_path}")

# Verify loaded model works
test_pred_original = best_model.predict(data['X_test'][:5])
test_pred_loaded = loaded_model.predict(data['X_test'][:5])

print(f"\n🔍 Verification:")
print(f"  Original predictions: {test_pred_original}")
print(f"  Loaded predictions:   {test_pred_loaded}")
print(f"  Predictions match: {np.array_equal(test_pred_original, test_pred_loaded)}")

In [ ]:
# Generate comprehensive explanation report
print("📋 Generating Comprehensive Explanation Report...")

comprehensive_report = explainer.generate_explanation_report(
    data['X_test'], 
    num_samples=3
)

print(f"\n📊 Report Summary:")
summary = comprehensive_report['summary']
for key, value in summary.items():
    print(f"  {key}: {value}")

print(f"\n🌍 Global Explanation Methods Available:")
for method in comprehensive_report['global_explanations'].keys():
    print(f"  • {method.upper()}")

print(f"\n🎯 Local Explanations Generated: {len(comprehensive_report['sample_explanations'])}")

# Show feature interactions summary
if 'feature_interactions' in comprehensive_report:
    interactions = comprehensive_report['feature_interactions']
    if isinstance(interactions, dict) and 'top_interactions' in interactions:
        print(f"\n🔗 Feature Interactions Analyzed: {len(interactions['top_interactions'])}")

In [ ]:
# Performance summary
print("🏆 XplainML Demo Summary")
print("=" * 50)

print(f"📊 Dataset:")
print(f"  • Samples: {data['original_shape'][0]}")
print(f"  • Features: {len(data['feature_names'])}")
print(f"  • Task: {data['task_type'].title()}")
print(f"  • Target: {data['target_name']}")

print(f"\n🤖 Best Model:")
print(f"  • Algorithm: {best_model.model_type.title()}")
print(f"  • Performance: {best_score:.4f} {metric}")

print(f"\n🔍 Explanations Generated:")
print(f"  • Global methods: {len(global_explanations)}")
print(f"  • Local samples: {len(local_explanations)}")
print(f"  • Feature interactions: {'✅' if isinstance(interactions, dict) else '❌'}")

print(f"\n📈 Visualizations Created:")
print(f"  • Feature importance plots")
print(f"  • Prediction distributions")
print(f"  • Partial dependence plots")
print(f"  • Model comparison charts")

print(f"\n🎯 Predictions:")
print(f"  • Test set accuracy: {test_metrics.get('accuracy', test_metrics.get('r2', 'N/A'))}")
print(f"  • Single predictions: ✅")
print(f"  • Batch predictions: ✅")
print(f"  • Confidence scoring: ✅")

print(f"\n✨ Advanced Features:")
print(f"  • Model saving/loading: ✅")
print(f"  • Hyperparameter tuning: ✅")
print(f"  • Model comparison: ✅")
print(f"  • Comprehensive reporting: ✅")

print(f"\n🎉 XplainML Demo Completed Successfully!")
print(f"🚀 Your machine learning models are now interpretable and explainable!")

## Conclusion

This notebook demonstrated the comprehensive capabilities of XplainML:

### ✅ What We Accomplished:

1. **Data Processing**: Loaded, cleaned, and preprocessed tabular data
2. **Model Training**: Trained multiple ML models and compared performance
3. **Hyperparameter Tuning**: Optimized model parameters automatically
4. **Model Explanations**: Generated global and local explanations using SHAP, LIME, and permutation importance
5. **Visualizations**: Created comprehensive plots for understanding model behavior
6. **Predictions**: Made single and batch predictions with confidence scores
7. **Advanced Features**: Explored feature interactions, model persistence, and comprehensive reporting

### 🎯 Key Benefits of XplainML:

- **Transparency**: Understand why your model makes specific predictions
- **Trust**: Build confidence in your ML models through explanations
- **Debugging**: Identify model weaknesses and areas for improvement
- **Compliance**: Meet regulatory requirements for explainable AI
- **Insights**: Discover important patterns in your data

### 🚀 Next Steps:

1. Try XplainML with your own datasets
2. Explore the web dashboard for interactive analysis
3. Use the CLI for automated workflows
4. Customize explanations for your specific use case
5. Share insights with stakeholders using generated reports

---

**Happy explaining! 🎉**